In [2]:
import torch

In [3]:
def TTSVD(X,r):
  
    d = X.shape
    m = len(d)
    

    C = X
    T = []
    C = C.reshape(d[0], -1)
    [U, S, V] = torch.linalg.svd(C,full_matrices=False)
    T.append(U[:,:r[0]])
    C = torch.diag(S[:r[0]])@V[:r[0],:]

    
    for i in range(1,m-1):
        # unfold the temporary tensor into a matrix with size r_n-1In x I_n+1...I_N 
        C = C.reshape(int(d[i]*r[i]), -1)
        # do the SVD for unfold C
        [U, S, V] = torch.linalg.svd(C,full_matrices=False)
        
        T.append(U[:, :r[i+1]].reshape(r[i],d[i],r[i+1]))
        # update the temporary tensor to matrix with size r_n x I_n+1...I_N
        C = torch.diag(S[:r[i+1]])@V[:r[i+1],:]
    
    T.append(C)
    
    return T


In [4]:
def TTSVD_2(X,r):
    C = X
    T_left = torch.ones(1)
    d = X.shape
    m = len(d)
    T = []

    d_prod = d[0]

    for i in range(0,m-1):
        T_i = C.reshape(d_prod,-1)
        temp = (torch.kron(T_left,torch.eye(d[i])))
        [U,S,V] = torch.linalg.svd(temp.T@T_i,full_matrices=False)
        L = U[:,:r[i]]
        if i == 0:
            T.append(L.reshape(d[i],r[i]))
        else:
            T.append(L.reshape(r[i-1],d[i],-1))
        T_left = temp@L
        d_prod = d_prod*d[i+1]
    
    T_m_1 = C.reshape(int(d_prod/d[m-1]),-1)
    T.append(T_left.T@T_m_1)

    return T

In [16]:
X = torch.randn(3,3,3)
r = [2,2,2]


In [17]:
print(TTSVD(X,r))

[tensor([[-0.2183,  0.8118],
        [-0.4400, -0.5772],
        [ 0.8710, -0.0882]]), tensor([[[-0.8501, -0.2848],
         [ 0.1957, -0.6583],
         [-0.3567,  0.2189]],

        [[-0.0817, -0.0548],
         [-0.0939,  0.6555],
         [-0.3104, -0.0703]]]), tensor([[-0.0924,  1.3723,  2.2959],
        [ 2.2821, -0.4781,  0.3776]])]


In [19]:
print(TTSVD_2(X,[2,2]))

[tensor([[-0.2183,  0.8118],
        [-0.4400, -0.5772],
        [ 0.8710, -0.0882]]), tensor([[[-0.8501, -0.2848],
         [ 0.1957, -0.6583],
         [-0.3567,  0.2189]],

        [[-0.0817, -0.0548],
         [-0.0939,  0.6555],
         [-0.3104, -0.0703]]]), tensor([[-0.0924,  1.3723,  2.2959],
        [ 2.2821, -0.4781,  0.3776]])]
